## <span style="color:orange">Nicolas Leguizamón, código: 201727960 </span>
## <span style="color:orange">Leidy Araque Molina, código: 201727196 </span>

 <h1> <center> <span style="color:orange"> FINAL PROJECT - TEXT MODELS</span></center></h1>

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Read Data

In [2]:
path = 'C:\\Users\\AsusPC\\Documents\\Analytics\\2018-Vacacional\\DeepLearning\\Project'

In [3]:
dataTraining = pd.read_csv(os.path.join(path, 'data', 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'data', 'dataTesting.csv'), encoding='UTF-8', index_col=0)

In [4]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In [5]:
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


In order to achieve better results with the present model we are going to use the feature extraction techniques that we learned in the NLP sessions, the idea is to vectorize the feature and extract them to simplify the model and make it more accurate.
Character n-grams are a very successful approach to represent text for stylistic purposes since they are able to capture nuances in lexical, syntactical, and structural level

# Create count vectorizer with ngrams

In [46]:
vect = CountVectorizer(ngram_range=(1, 2), max_features=1000)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1000)

In [47]:
print(vect.get_feature_names()[:50])

['able', 'able to', 'about', 'about the', 'about to', 'accident', 'across', 'act', 'action', 'actually', 'affair', 'after', 'after the', 'again', 'against', 'against the', 'age', 'agent', 'ago', 'alex', 'alive', 'all', 'all of', 'all the', 'alone', 'along', 'along the', 'along with', 'already', 'also', 'although', 'always', 'america', 'american', 'among', 'an', 'an old', 'and', 'and has', 'and he', 'and her', 'and his', 'and is', 'and she', 'and that', 'and the', 'and their', 'and then', 'and they', 'and when']


In [48]:
X_dtm

<7895x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 470792 stored elements in Compressed Sparse Row format>

# Create y

In [10]:
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))

le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

# Split train and test

In [39]:
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [40]:
X_train

<5289x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 316874 stored elements in Compressed Sparse Row format>

#  <span style="color:orange">1. Machine Learning algorithm </span>

# Train multi-class multi-label model

In [35]:
#clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))

In [36]:
#clf = OneVsRestClassifier(BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)) 

In [187]:
from sklearn.ensemble import ExtraTreesClassifier
clf = OneVsRestClassifier(ExtraTreesClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))

In [188]:
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=10, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          n_jobs=1)

In [189]:
y_pred_genres = clf.predict_proba(X_test)

In [190]:
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7569697472601943

In [191]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = clf.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_text_mach_1.csv', index_label='ID')

#  <span style="color:orange">2. Deep Learning algorithm </span>

In [6]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Lambda
from keras.optimizers import RMSprop
from keras.callbacks import History
#from livelossplot import PlotLossesKeras
from keras import backend as K

C:\Users\AsusPC\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Another approach is to use the count vectorizer function, we consider these tachniques can be of use in this case since the text is more sucinct and have not so many sinctactical features that can require sophisticated treatment such as in phrases or conversations, also it does not reflect any sentiment that can introduce any Bias or missclassification. 

##  CountVectorizer

In [192]:
vect = CountVectorizer(lowercase=False, stop_words='english', max_features=1500) #,ngram_range=(1, 2)
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 1500)

In [166]:
from keras.layers.recurrent import LSTM

In [193]:
model = Sequential()
model.add(Dense(24,input_shape=X_dtm.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(24))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [194]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 24)                36024     
_________________________________________________________________
activation_23 (Activation)   (None, 24)                0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 24)                96        
_________________________________________________________________
dropout_17 (Dropout)         (None, 24)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 24)                600       
_________________________________________________________________
activation_24 (Activation)   (None, 24)                0         
Total params: 36,720
Trainable params: 36,672
Non-trainable params: 48
_________________________________________________________________


In [195]:
model.fit(X_dtm, y_genres, epochs=50, verbose = 1)


Epoch 1/50
7895/7895 [==============================] - 3s 388us/step - loss: 8.7690 - acc: 0.0509
Epoch 2/50
7895/7895 [==============================] - 1s 132us/step - loss: 8.2945 - acc: 0.1132
Epoch 3/50
7895/7895 [==============================] - 1s 138us/step - loss: 8.0006 - acc: 0.1738
Epoch 4/50
7895/7895 [==============================] - 1s 132us/step - loss: 7.7616 - acc: 0.2149
Epoch 5/50
7895/7895 [==============================] - 1s 142us/step - loss: 7.5208 - acc: 0.2421
Epoch 6/50
7895/7895 [==============================] - 1s 130us/step - loss: 7.2972 - acc: 0.2441
Epoch 7/50
7895/7895 [==============================] - 1s 141us/step - loss: 7.1176 - acc: 0.2612
Epoch 8/50
7895/7895 [==============================] - 1s 137us/step - loss: 6.9540 - acc: 0.2586
Epoch 9/50
7895/7895 [==============================] - 1s 135us/step - loss: 6.8305 - acc: 0.2509
Epoch 10/50
7895/7895 [==============================] - 1s 132us/step - loss: 6.7212 - acc: 0.2514
Epoch 11/

In [197]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_text_countVec.csv', index_label='ID')

With the previous model was obtained in <span style="color:blue">KAGGLE 0.8259</span>

this approach yield to a considerable high score, we can see if we can improve it by means of using anther approach, we know that LSTM can be more suitable for time series such as voice.

## TfidfVectorizer

TFID is very useful to extarct features and give the weight to those that resut more frequent, we are going to USE this approach to enhance the model

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# create a document-term matrix using TF-IDF
vect = TfidfVectorizer(stop_words='english')
X_dtm = vect.fit_transform(dataTraining['plot'])

In [31]:
model = Sequential()
model.add(Dense(24,input_shape=X_dtm.shape[1:]))
model.add(Activation('tanh'))
model.add(BatchNormalization())
model.add(Dropout(0.7))
model.add(Dense(24))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [32]:
model.fit(X_dtm, y_genres, epochs=50, verbose = 1)

Epoch 1/50
7895/7895 [==============================] - 11s 1ms/step - loss: 8.0780 - acc: 0.1567
Epoch 2/50
7895/7895 [==============================] - 10s 1ms/step - loss: 7.0518 - acc: 0.2894
Epoch 3/50
7895/7895 [==============================] - 10s 1ms/step - loss: 6.6135 - acc: 0.3326
Epoch 4/50
7895/7895 [==============================] - 10s 1ms/step - loss: 6.3233 - acc: 0.3564
Epoch 5/50
7895/7895 [==============================] - 9s 1ms/step - loss: 6.1115 - acc: 0.3745
Epoch 6/50
7895/7895 [==============================] - 9s 1ms/step - loss: 5.9386 - acc: 0.3911
Epoch 7/50
7895/7895 [==============================] - 9s 1ms/step - loss: 5.8255 - acc: 0.3902
Epoch 8/50
7895/7895 [==============================] - 10s 1ms/step - loss: 5.7347 - acc: 0.3914
Epoch 9/50
7895/7895 [==============================] - 9s 1ms/step - loss: 5.6637 - acc: 0.4006
Epoch 10/50
7895/7895 [==============================] - 10s 1ms/step - loss: 5.5713 - acc: 0.4157: 1s - 
Epoch 11/50
7895

In [33]:
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict_proba(X_test_dtm)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_text_TfidV.csv', index_label='ID')

With the previous model was obtained in <span style="color:blue">KAGGLE 0.85603</span>

effectively; the approach leads to a better model we are going to keep this in order to average the two models into a single one

## Using preprocesing vocabulary

In [207]:
top_words = 5000
index_from = 3
max_review_length = 500

In [208]:
from keras.datasets.imdb import get_word_index

vocab = get_word_index()

vocab = {k:(v+index_from) for k,v in vocab.items()}
vocab["<PAD>"] = 0
vocab["<START>"] = 1
vocab["<UNK>"] = 2

In [209]:
{k:vocab[k] for k in list(vocab.keys())[:10]}

{'fawn': 34704,
 'hanging': 2348,
 'nunnery': 52010,
 'sonja': 16819,
 'spiders': 16118,
 'trawling': 52011,
 'tsukino': 52009,
 'vani': 63954,
 'woods': 1411,
 'woody': 2292}

## Training

In [177]:
reviews = [x[:max_review_length] for x in dataTraining['plot']]
X=reviews
# Convert characters to int and pad
X_voc = [[vocab[x1] for x1 in x if x1 in vocab.keys()] for x in X]

# Testing

In [178]:
reviews = [x[:max_review_length] for x in dataTesting['plot']]
X=reviews
# Convert characters to int and pad
X_test_k = [[vocab[x1] for x1 in x if x1 in vocab.keys()] for x in X]

In [98]:
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from livelossplot import PlotLossesKeras
%matplotlib inline

In [179]:
X_voc_pad = sequence.pad_sequences(X_voc, maxlen=max_review_length)
X_test_k_pad = sequence.pad_sequences(X_test_k, maxlen=max_review_length)

In [180]:
X_voc_pad[X_voc_pad>5000]=2
X_test_k_pad[X_test_k_pad>5000]=2

In [183]:
model = Sequential()
#embedding es disminucion de dimensionalidad
model.add(Embedding(top_words + 1, 32, input_length=max_review_length))
model.add(LSTM(100))
model.add(BatchNormalization())
model.add(Dropout(0.8))
model.add(Dense(24, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 500, 32)           160032    
_________________________________________________________________
lstm_12 (LSTM)               (None, 100)               53200     
_________________________________________________________________
batch_normalization_16 (Batc (None, 100)               400       
_________________________________________________________________
dropout_16 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 24)                2424      
Total params: 216,056
Trainable params: 215,856
Non-trainable params: 200
_________________________________________________________________


In [185]:
model.fit(X_voc_pad, y_genres, epochs=20, verbose = 2)


Epoch 1/20
 - 146s - loss: 7.4838 - acc: 0.1976
Epoch 2/20
 - 147s - loss: 7.4683 - acc: 0.1980
Epoch 3/20
 - 145s - loss: 7.4666 - acc: 0.1977
Epoch 4/20
 - 148s - loss: 7.4601 - acc: 0.1977
Epoch 5/20
 - 144s - loss: 7.4604 - acc: 0.1980
Epoch 6/20
 - 140s - loss: 7.4597 - acc: 0.1978
Epoch 7/20
 - 149s - loss: 7.4607 - acc: 0.1975
Epoch 8/20
 - 145s - loss: 7.4613 - acc: 0.1978
Epoch 9/20
 - 146s - loss: 7.4488 - acc: 0.1978
Epoch 10/20
 - 151s - loss: 7.4554 - acc: 0.1978
Epoch 11/20
 - 183s - loss: 7.4546 - acc: 0.1977
Epoch 12/20
 - 192s - loss: 7.4519 - acc: 0.1978
Epoch 13/20
 - 194s - loss: 7.4554 - acc: 0.1980
Epoch 14/20
 - 197s - loss: 7.4520 - acc: 0.1980
Epoch 15/20
 - 176s - loss: 7.4559 - acc: 0.1978
Epoch 16/20
 - 171s - loss: 7.4574 - acc: 0.1976
Epoch 17/20
 - 142s - loss: 7.4525 - acc: 0.1978
Epoch 18/20
 - 156s - loss: 7.4449 - acc: 0.1978
Epoch 19/20
 - 147s - loss: 7.4552 - acc: 0.1980
Epoch 20/20
 - 145s - loss: 7.4547 - acc: 0.1980


#  <span style="color:orange">3. Average with Image model and Text model </span>

what we did here is to use the model outputs to make the weighted average of the two models (giving a higher consideration of the text model since it has better performance.


In [27]:
pred_img = pd.read_csv(os.path.join(path, 'pred_genres_img_RF.csv'), encoding='UTF-8', index_col=0)
pred_text = pd.read_csv(os.path.join(path, 'pred_text_TfidV.csv'), encoding='UTF-8', index_col=0)

In [28]:
prediction=0.3*pred_img+0.7*pred_text

In [29]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

pd.DataFrame(prediction, index=dataTesting.index, columns=cols).to_csv('average.csv', index_label='ID')